In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pymongo

In [2]:
def finding_raw_urls_from_base(url):
    service = Service("./chromedriver/chromedriver.exe")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'a'))
    )
    
    html = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', href=True)
    raw_urls = []
    for link in links:
        if link['href'].startswith("https://www.moneycontrol.com/news/business/"):
            raw_urls.append(link['href'])
    return list(set(raw_urls))
    

In [3]:
# raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/")
raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/page-2")

In [4]:
raw_urls

['https://www.moneycontrol.com/news/business/commodities/',
 'https://www.moneycontrol.com/news/business/standard-glass-lining-ipo-gmp-live-capital-infra-trust-quadrant-future-tek-ipo-gmp-business-news-january-8-liveblog-12905363.html',
 'https://www.moneycontrol.com/news/business/ipo/',
 'https://www.moneycontrol.com/news/business/markets/bls-international-says-december-travels-underwhelming-but-confident-of-maintaining-margins-12905754.html',
 'https://www.moneycontrol.com/news/business/personal-finance/income-tax-filing/',
 'https://www.moneycontrol.com/news/business/banks/',
 'https://www.moneycontrol.com/news/business/markets/premier-waaree-energies-share-slide-2-as-kotak-institutional-equities-initiates-with-sell-12905535.html',
 'https://www.moneycontrol.com/news/business/markets/ongc-picks-bp-arm-to-help-ramp-up-mumbai-high-production-sees-60-more-output-12905534.html',
 'https://www.moneycontrol.com/news/business/markets/stock-market-live-sensex-nifty-50-share-price-gift-nifty

In [5]:
stock_urls =[]
market_urls = []
next_page_urls = []
other_urls = []
ipo_urls = []
def extract_urls(all_urls):
    for link in raw_urls:
        if link.startswith("https://www.moneycontrol.com/news/business/stocks/page"):
            next_page_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/stocks/"):
            stock_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/markets/"):
            market_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/ipo/"):
            ipo_urls.append(link)
        else:
            other_urls.append(link)

In [6]:
extract_urls(raw_urls)

In [7]:
stock_urls

['https://www.moneycontrol.com/news/business/stocks/top-gainers-and-losers-today-stocks-that-moved-the-most-on-january-8-12905881.html',
 'https://www.moneycontrol.com/news/business/stocks/suven-life-shares-rise-11-on-progress-in-clinical-trial-of-cognitive-disorder-drug-12905778.html',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-08-2025-geojit-financial-services-12905424.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-jnk-india-target-of-rs-815-lkp-research-12905515.html',
 'https://www.moneycontrol.com/news/business/stocks/signatureglobal-india-shares-gain-after-firm-reports-strong-fy25-pre-sales-12905442.html',
 'https://www.moneycontrol.com/news/business/stocks/',
 'https://www.moneycontrol.com/news/business/stocks/tata-steel-shares-trade-marginally-lower-post-december-quarter-business-update-12905775.html']

In [8]:

regex = r'https:\/\/www\.moneycontrol\.com\/news\/business\/stocks\/[^\/\s]+(?:\.[a-z]{2,6})(?:[\/\?].*)?'

final_stocks_urls = [url for url in stock_urls if re.match(regex, url)]

In [9]:
final_stocks_urls

['https://www.moneycontrol.com/news/business/stocks/top-gainers-and-losers-today-stocks-that-moved-the-most-on-january-8-12905881.html',
 'https://www.moneycontrol.com/news/business/stocks/suven-life-shares-rise-11-on-progress-in-clinical-trial-of-cognitive-disorder-drug-12905778.html',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-08-2025-geojit-financial-services-12905424.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-jnk-india-target-of-rs-815-lkp-research-12905515.html',
 'https://www.moneycontrol.com/news/business/stocks/signatureglobal-india-shares-gain-after-firm-reports-strong-fy25-pre-sales-12905442.html',
 'https://www.moneycontrol.com/news/business/stocks/tata-steel-shares-trade-marginally-lower-post-december-quarter-business-update-12905775.html']

In [10]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["new_db"]
mycol = mydb['raw_news']

In [11]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")

driver = webdriver.Chrome(service=service, options= options)

In [12]:
def extract_data(url):  
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    news = {}
    
    title = soup.find('h1', class_ = "article_title")
    news.update({"title":f"{title.text.strip()}"})
    
    desc = soup.find('h2', class_ = 'article_desc')
    news.update({"desc":f"{desc.text.strip()}"})
    
    date_time_div = soup.find('div', class_="article_schedule")
    if date_time_div:
        span_tag = date_time_div.find('span')
        date = span_tag.text.strip()
        news.update({"date":f"{date}"})
    
    datetime = date_time_div.text.strip()
    news.update({"datetime":f"{datetime}"})
    
    paragraphs_list = []
    paragrphs_div = soup.find('div', class_ ="content_wrapper")
    if paragrphs_div:
        paragraph_tags = paragrphs_div.find_all('p')
        for p in paragraph_tags:
            para_text = p.text.strip()            
            if len(para_text) < 50 :
                continue            
            if re.search(r"(click\s+here|disclaimer|modal|window|advertisement|investment\s+tips)", para_text, re.IGNORECASE):
                continue            
            paragraphs_list.append(para_text)         
            
    news.update({"content": paragraphs_list})
    
    stock_name = soup.find('a', class_="stock-name")
    if stock_name:
        news.update({"stock_name":f"{stock_name.text.strip()}"})
            
    # driver.quit()   
    return news

In [13]:
for url in final_stocks_urls :
    data = extract_data(url)
    x = mycol.insert_one(data)
    print(x.inserted_id)

677f5e35445f7dea037d72df


677f5e56445f7dea037d72e0


677f5e60445f7dea037d72e1


677f5e66445f7dea037d72e2


677f5e6a445f7dea037d72e3


677f5e75445f7dea037d72e4


In [14]:
driver.quit()   